In [7]:
from plotnine import *
import polars as pl

In [8]:
df = pl.read_csv("teams_per_year.csv")

In [9]:
df

Year,NYK,ATL,LAL,SAC,DET,GSW,PHI,BOS,WAS,HOU,OKC,MIL,PHO,CHI,CLE,LAC,POR,UTA,NJN,SAS,IND,DEN,DAL,CHA,ORL,MIA,MIN,TOR,MEM,NOH
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""2025""",1,0,1,1,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,1,1,0,0,1,1,1,1,1
"""2024""",1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1
"""2023""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"""2022""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"""2021""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1953""",1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""1952""",1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""1951""",1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# Plot the Total row for every column
totals = df.filter(pl.col("Year") == "Total").select(pl.col(pl.Int64)).unpivot()
plot = (
    ggplot(totals)
    + aes(x="variable", y="value")
    + geom_col()
    + labs(title="Total Years per Team", x="Team", y="Number of Years")
    # Rotate x labels
    + theme(axis_text_x=element_text(rotation=90, hjust=1))
    # Sort by value
    + scale_x_discrete(limits=totals.sort("value", descending=True).select("variable").to_series().to_list())
    # Add value labels to each bar
    + geom_text(aes(label="value"), va="bottom", size=8, format_string="{:.0f}")
)

In [11]:
plot

# Save the plot
plot.save("teams_per_year.png", width=10, height=6, dpi=500)

/Users/stefanbroecker/miniforge3/envs/nba/lib/python3.13/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 10 x 6 in image.
/Users/stefanbroecker/miniforge3/envs/nba/lib/python3.13/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: teams_per_year.png


In [ ]:
# Make a heatmap where the x-axis is the team, the y-axis is the year, and the color red or green for 0 and 1

teams = df.columns[1:]  # Assuming the first column is 'year' and the rest are teams

# Drop the totals row (the last row)
df = df.filter(pl.col("Year") != "Total")

# Convert to long format for plotting
df_long = df.unpivot(
    index=['Year'],
    on=teams,
    variable_name='team',
    value_name='value'
)

# Create the heatmap
plot = (
    ggplot(df_long, aes(x='team', y='Year', fill='factor(value)'))
    # Add white borders between tiles
    + geom_tile(color='white', size=0.5) 
    # Color the tiles
    + scale_fill_manual(
        values={0: 'red', 1: 'green'},
        name='Had a Player',
        labels=['No', 'Yes']
    )
    + scale_x_discrete(limits=teams) # Order teams by the total years
    + theme_minimal() 
    # Add labels and title
    + labs(
        title='Teams in the Finals Heatmap',
        x='Teams',
        y='Years',
        fill='Value'
    ) 
    + theme(
        axis_text_x=element_text(angle=45),  # Rotate x-axis labels
        figure_size=(12, 8)
    )
)

# Display the plot
plot

# Save the plot
plot.save('heatmap.png', dpi=500)

/Users/stefanbroecker/miniforge3/envs/nba/lib/python3.13/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 12 x 8 in image.
/Users/stefanbroecker/miniforge3/envs/nba/lib/python3.13/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: heatmap.png
